# FINAL PROJECT TLCN - CHATBOT HỖ TRỢ TRA CỨU THÔNG TIN SỨC KHỎE 
- Author:
  - Đặng Kim Thành - 21110298
  - Bùi Quốc Khang - 21110202
## Phần 3 - MeDiChat ViVi - Retrieval-Augmented Generation Advancde

### I. Xử lý dữ liệu

#### Tải môi trường thư viện và kiểm tra GPU

In [1]:
import unidecode
import re
import pdfplumber
import os
import pickle
import json

In [2]:
import torch
print(torch.cuda.is_available())

True


#### Xử lý tên file của dữ liệu (đề phòng trường hợp lỗi khi tải dữ liệu)

In [5]:
folder_path = 'Data/PDF'  # Đường dẫn tới thư mục chứa các tệp cần đổi tên

In [5]:
# Duyệt qua tất cả các tệp trong thư mục
for filename in os.listdir(folder_path):
    # Chuyển đổi tên tệp sang không dấu
    new_filename = unidecode.unidecode(filename)
    # Thay thế dấu cách bằng dấu gạch dưới
    
    new_filename = new_filename.replace(' ', '_')
    # Loại bỏ các ký tự đặc biệt, chỉ giữ lại chữ cái, số và dấu gạch dưới
    new_filename = re.sub(r'[^A-Za-z0-9_.]', '', new_filename)

    # Đổi tên tệp
    os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))
    print(f"Đã đổi tên tệp: {filename} -> {new_filename}")

Đã đổi tên tệp: co_che_trieu_chung_hoc.pdf -> co_che_trieu_chung_hoc.pdf


#### Xử lý dữ liệu đầu vào PDF thành JSON

In [2]:
# Normalize filenames for consistency
def normalize_filename(filename):
    filename = unidecode.unidecode(filename)
    filename = filename.replace(" ", "_")
    return filename

# Remove page numbers from the text
def remove_page_numbers(text):
    # Remove page numbers at the start or end of a line (assume numbers at the beginning or end are page numbers)
    text = re.sub(r'^\s*\d+\s*$', '', text.strip())  # Remove page numbers at the start of the line
    text = re.sub(r'\s*\d+\s*$', '', text.strip())  # Remove page numbers at the end of the line
    return text.strip()

# Clean and merge fragmented lines
def clean_and_merge_lines(lines):
    merged_lines = []
    buffer = ""
    for line in lines:
        stripped_line = line.strip()
        if buffer:
            if stripped_line and not stripped_line[0].isupper():
                buffer += " " + stripped_line  # Merge fragmented lines
            else:
                merged_lines.append(buffer)
                buffer = stripped_line
        else:
            buffer = stripped_line
    if buffer:
        merged_lines.append(buffer)
    return merged_lines

# Process content and extract disease information (title and description)
def process_disease_content(lines, skip_first_line=False, title="", previous_title=""):
    disease_data = {
        "title": "",
        "description": "",
        "causes": "",
        "mechanism": "",
        "meaning": ""
    }
    section = ""
    current_section = None

    # Nếu cần bỏ qua dòng đầu tiên (thường là tiêu đề)
    if skip_first_line and len(lines) > 0:
        lines = lines[1:]  # Bỏ qua dòng đầu tiên

        # Xử lý loại bỏ số trang nếu có ở dòng đầu tiên sau khi bỏ qua
        if len(lines) > 0:
            lines[0] = re.sub(r'\s*\d+\s*$', '', lines[0])  # Loại bỏ số trang ở cuối dòng
            lines[0] = re.sub(r'^\s*\d+\s*', '', lines[0])  # Loại bỏ số trang ở đầu dòng

    # Nếu tiêu đề trùng với dòng đầu tiên, xóa dòng đầu tiên
    if len(lines) > 0 and remove_page_numbers(lines[0]) == title:
        lines = lines[1:]  # Xóa dòng đầu tiên nếu nó trùng với tiêu đề

    # Kiểm tra các phần mô tả (description, causes, mechanism, meaning)
    for line in lines:
        if 'MÔ TẢ' in line:
            current_section = "description"
        elif 'NGUYÊN NHÂN' in line:
            current_section = "causes"
        elif 'CƠ CHẾ' in line:
            current_section = "mechanism"
        elif 'Ý NGHĨA' in line:
            current_section = "meaning"
        
        # Bỏ qua tiêu đề lặp lại trong phần mô tả nếu đã xử lý
        if current_section == "description" and title == previous_title:
            line = re.sub(r'^\s*' + re.escape(title) + r'\s*', '', line)  # Xóa tiêu đề lặp lại trong phần mô tả

        # Lưu dữ liệu vào các phần tương ứng
        if current_section:
            if current_section == "description":
                disease_data["description"] += line + " "
            elif current_section == "causes":
                disease_data["causes"] += line + " "
            elif current_section == "mechanism":
                disease_data["mechanism"] += line + " "
            elif current_section == "meaning":
                disease_data["meaning"] += line + " "

    return disease_data

# Extract and process text and tables from a PDF
def pdf_to_json(pdf_file):
    data = {"pages": []}  # Store structured data for each page
    current_disease = None  # To keep track of the current disease being processed
    page_processed = set()  # To keep track of the pages that have already been processed
    previous_title = ""  # Track the previous title to avoid repetition

    with pdfplumber.open(pdf_file) as pdf:
        disease_data = []
        page_num = 0

        while page_num < len(pdf.pages):
            # Skip pages 2 to 12 (0-based indexing, so 2 to 12 means pages 3 to 13 are skipped)
            if 1 <= page_num <= 12:
                page_num += 1
                continue

            page = pdf.pages[page_num]
            page_content = {"title": "", "text": [], "tables": []}
            # Extract text
            text = page.extract_text()
            if text:
                text = remove_page_numbers(text)
                text_lines = text.split("\n")
                cleaned_lines = clean_and_merge_lines(text_lines)

                # Get the title from the first line (skip the page number or irrelevant info)
                if len(cleaned_lines) > 1:
                    title_with_page = cleaned_lines[0]  # First line might contain title with page number
                    title = re.sub(r'\s*\d+\s*$', '', title_with_page)  # Remove page number at the end
                    title = re.sub(r'^\s*\d+\s*', '', title)  # Remove page number at the start

                    page_content["title"] = title

                    # Check if this is a new disease or continuation
                    if current_disease is None or title != current_disease["title"]:
                        # New disease, extract data
                        if current_disease:
                            disease_data.append(current_disease)  # Save previous disease if any
                        current_disease = {
                            "title": title,
                            "description": "",
                            "causes": "",
                            "mechanism": "",
                            "meaning": ""
                        }

                    # Process the current page's content for the disease
                    disease_info = process_disease_content(cleaned_lines[1:], skip_first_line=True, title=title, previous_title=previous_title)
                    current_disease["description"] += disease_info["description"]
                    current_disease["causes"] += disease_info["causes"]
                    current_disease["mechanism"] += disease_info["mechanism"]
                    current_disease["meaning"] += disease_info["meaning"]

                    previous_title = title  # Update previous title

            # Extract tables (if necessary)
            tables = page.extract_tables()
            for table in tables:
                cleaned_table = [
                    [cell if cell else "" for cell in row] for row in table
                ]
                page_content["tables"].append(cleaned_table)

            page_content["text"] = cleaned_lines
            data["pages"].append(page_content)

            # Check if the next page has the same title (ignoring page numbers)
            if page_num + 1 < len(pdf.pages):
                next_page = pdf.pages[page_num + 1]
                next_text = next_page.extract_text()
                if next_text:
                    next_lines = next_text.split("\n")
                    next_cleaned_lines = clean_and_merge_lines(next_lines)
                    if len(next_cleaned_lines) > 1:
                        next_title = re.sub(r'\s*\d+\s*$', '', next_cleaned_lines[0])  # Remove page number
                        next_title = re.sub(r'^\s*\d+\s*', '', next_title)

                        if title == next_title:
                            page_num += 1  # Skip to the next page if the title is the same

            page_processed.add(page_num)
            page_num += 1

        # Add the last disease to the final data
        if current_disease:
            disease_data.append(current_disease)

        # Add the extracted disease data to the final output
        data["diseases"] = disease_data

    return data

# Save JSON data
def save_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# Process PDFs in a folder and save JSON files
def convert_pdfs_to_json(pdf_path, json_path):
    os.makedirs(json_path, exist_ok=True)
    for filename in os.listdir(pdf_path):
        if filename.endswith('.pdf'):
            normalized_filename = normalize_filename(filename)
            pdf_file = os.path.join(pdf_path, filename)
            json_file = os.path.join(json_path, normalized_filename.replace('.pdf', '.json'))
            structured_data = pdf_to_json(pdf_file)
            save_json(structured_data, json_file)
            print(f"Processed {pdf_file} and saved as {json_file}")

In [6]:
# Paths for input and output
pdf_path = "Data/PDF"  # Replace with your PDF folder path
json_path = "Data/JSON"  # Replace with your desired output folder for JSON

# Run the conversion
convert_pdfs_to_json(pdf_path, json_path)

Processed Data/PDF\co_che_trieu_chung_hoc.pdf and saved as Data/JSON\co_che_trieu_chung_hoc.json


#### Chuẩn hóa dữ liệu thành dạng Document của LangChain

In [3]:
from langchain.schema import Document

# Đọc tệp JSON
def load_json_data(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        return json.load(f)

# Chuyển dữ liệu từ JSON thành định dạng Document của LangChain
def convert_json_to_documents(json_data):
    documents = []

    # Duyệt qua các disease trong dữ liệu JSON
    for disease in json_data.get("diseases", []):
        title = disease.get("title", "")
        description = disease.get("description", "")
        causes = disease.get("causes", "")
        mechanism = disease.get("mechanism", "")
        meaning = disease.get("meaning", "")

        # Tạo content và metadata cho mỗi Document
        content = description + "\n" + causes + "\n" + mechanism + "\n" + meaning
        metadata = {
            "title": title,
            "table": disease.get("tables", [])
        }

        # Tạo Document
        document = Document(page_content=content, metadata=metadata)
        documents.append(document)

    return documents

# Hàm để load tất cả các tệp JSON trong thư mục
def load_all_json_files_in_directory(directory_path):
    documents = []
    # Duyệt qua tất cả các tệp trong thư mục
    for filename in os.listdir(directory_path):
        # Kiểm tra nếu tệp có phần mở rộng là .json
        if filename.endswith('.json'):
            json_file_path = os.path.join(directory_path, filename)
            # Đọc và chuyển tệp JSON thành documents
            json_data = load_json_data(json_file_path)
            documents.extend(convert_json_to_documents(json_data))
            print(f"Loaded: {filename}")
    return documents

In [4]:
# Đường dẫn tới thư mục chứa các tệp JSON
json_directory = "Data/JSON"  
# Load tất cả các tệp JSON trong thư mục
documents = load_all_json_files_in_directory(json_directory)

Loaded: co_che_trieu_chung_hoc.json


In [5]:
# In ra 1 document để kiểm tra
print(documents[6].page_content)
print(documents[6].metadata)

MÔ TẢ không phải là một nghiệm pháp chẩn đoán Bệnh nhân nằm sấp, đầu gối gấp 90°, tiến đặc biệt hữu ích của chấn thương sụn hành ép mạnh vào gót chân từ trên xuống chêm. Nhưng những phát hiện đã được dưới, ép xương chày xuống xương đùi. Sau nảy sinh trong kết quả của một nghiên cứu gộp. Ngoài ra, nhiều người khám cũng đó người thực hiện tiến hành xoay xương không thực hiện được nghiệm pháp xoay chày vào trong hoặc ra ngoài. của Apley do có thể gây đau dữ dội nếu xuất hiện kèm theo một chấn thương. 
NGUYÊN NHÂN • Chấn thương sụn chêm 
CƠ CHẾ Áp lực lớn từ xương chày lên xương đùi sẽ được nhằm vào khe khớp tại vị trí gây tổn thương sụn chêm. Nếu có tổn thương trực tiếp có thể gây nên đau. 
Ý NGHĨA Một vài nghiên cứu không đồng nhất đã được hoàn thành. Kết quả tổng hợp của hệ thống bảy nghiên cứu cho thấy độ nhạy của nghiệm pháp là 60.7% và độ đặc hiệu là 70,2% với một tỷ lệ không đồng đều HÌNH 1.2 Nghiệm pháp Apley 3.4, Làm nghiệm pháp xoay của Apley 
{'title': 'Nghiệm pháp Apley', 'tabl

### II. Vector hóa và lưu dữ liệu

#### Semantic Chunking and Data split

In [6]:
from transformers import AutoTokenizer

# Load the tokenizer for the BAAI/bge-m3 model
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")

c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

# Khởi tạo HuggingFaceEmbeddings với model BAAI/bge-base-en-v1.5
model_name = "BAAI/bge-base-en-v1.5"
# model_kwargs = {'device': 'cpu'}
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False} # Chuẩn hóa vector = True sẽ limit length vector = 1
hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )

text_splitter = SemanticChunker(hf_embeddings,
                                breakpoint_threshold_type="percentile",
                                breakpoint_threshold_amount=85)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_37384\1580162735.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(


In [21]:
# Split the documents and count the number of chunks in each document, as well as the number of tokens in each chunk
count = 0
total = 0
list_of_chunks = []
for idx,doc in enumerate(documents):
  chunks = text_splitter.create_documents([doc.page_content])
  print(f'Number of chunks: {len(chunks)} - Tokens of each chunk',end=' ')
  for chunk in chunks:
      text = chunk.page_content
      tokens = tokenizer.tokenize(text)
      num_tokens = len(tokens)
      if num_tokens > 1:
        total = total + 1
        # Use the parent document index as metadata to retrieve the parent document from the child chunk.
        chunk.metadata['parent'] = idx
        list_of_chunks.append(chunk)
      if num_tokens > 512:
        count = count + 1
      print(num_tokens, end =' ')
  print()
print('Toltal chunks: ',total)
print('Number of chunks which is larger than 512 tokens: ',count)

Number of chunks: 1 - Tokens of each chunk 0 
Number of chunks: 2 - Tokens of each chunk 109 8 
Number of chunks: 2 - Tokens of each chunk 136 48 
Number of chunks: 1 - Tokens of each chunk 0 
Number of chunks: 1 - Tokens of each chunk 0 
Number of chunks: 2 - Tokens of each chunk 341 17 
Number of chunks: 3 - Tokens of each chunk 68 123 71 
Number of chunks: 2 - Tokens of each chunk 274 0 
Number of chunks: 3 - Tokens of each chunk 109 221 113 
Number of chunks: 4 - Tokens of each chunk 258 365 42 25 
Number of chunks: 3 - Tokens of each chunk 91 287 117 
Number of chunks: 2 - Tokens of each chunk 275 22 
Number of chunks: 3 - Tokens of each chunk 111 366 0 
Number of chunks: 4 - Tokens of each chunk 458 57 108 0 
Number of chunks: 3 - Tokens of each chunk 83 43 356 
Number of chunks: 4 - Tokens of each chunk 381 20 228 71 
Number of chunks: 2 - Tokens of each chunk 85 5 
Number of chunks: 3 - Tokens of each chunk 305 18 208 
Number of chunks: 1 - Tokens of each chunk 0 
Number of chu

In [22]:
# Chunk example
list_of_chunks[-1]

Document(metadata={'parent': 450}, page_content='Cơ chế chưa rõ. Tronghộichứng Turner, Trong hội chứng Turner, có đến thiếu một phần hay toàn bộ một nhiễm sắc 40% bệnh nhân sẽ có triệu chứng này. thể giới tính, tuy nhiên không rõ tại sao điều này lại dẫn đến triệu chứng cổ có màng. Ý NGHĨA toàn bộ 1 nhiễm sắc thể giới tính. Một dấu hiệu ít gặp và nếu thật sự hiện hữu • HộichứngNoonan– đột biến gen thì triệu chứng này luôn mang ý nghĩa bệnh ')

#### Lưu chuck for bm25 (Chỉ chạy khi chưa lưu dữ liệu lại)

In [23]:
# Lưu dữ liệu Documents ở lần chạy đầu tiên
with open("vectorstore/db_document/documents.pkl", "wb") as f:
    pickle.dump(list_of_chunks, f)

print("All documents saved successfully!")

All documents saved successfully!


#### Load chuck for bm25 (Chạy khi đã lưu dữ liệu)

In [8]:
# Tải lại tất cả các Document từ tệp pickle
with open("vectorstore/db_document/documents.pkl", "rb") as f:
    list_of_chunks = pickle.load(f)

print("All documents loaded successfully!")
print(list_of_chunks)

All documents loaded successfully!
[Document(metadata={'parent': 1}, page_content='Trong từng Triệu Chứng, các Tiêu Đề được thiết kế một cách đồng bộ:MÔ TẢ, \nNGUYÊN NHÂN, CƠ CHẾ, Ý NGHĨArất dễ dàng cho việc tra cứu. Cơ chế của từng triệu chứng được giải thích ngắn gọn nhưng chính xác, đầy đủ, giúp cho người đọc dễ hiểu Cuốn sách này thực sự là cuốn sách giá trị đối với tất cả các học viên ở bất kì trình độ nào, từ sinh viên Y khoa năm nhất tới các bác sĩ trẻ mới tốt nghiệp.'), Document(metadata={'parent': 1}, page_content='Giáo Sư Chris Semsarian xv \n\n'), Document(metadata={'parent': 2}, page_content='Trong từng Triệu Chứng, các Tiêu Đề được thiết kế một cách đồng bộ:MÔ TẢ, \nNGUYÊN NHÂN, CƠ CHẾ, Ý NGHĨArất dễ dàng cho việc tra cứu. Cơ chế của từng triệu chứng được giải thích ngắn gọn nhưng chính xác, đầy đủ, giúp cho người đọc dễ hiểu Cuốn sách được hòan thành là nhờ sự cố gắng rất lớn của nhóm dịch ‘’Chia sẻ Ca Lâm Sàng’’ với trưởng nhóm là Admin Fanpage : Chia Sẻ Ca Lâm Sàng. Cuố

#### Thiết lập id cho các chunks

In [25]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(list_of_chunks))]

In [26]:
uuids

['6ab9d26c-3ff1-4b82-9d2e-7d1bf506f170',
 '282b6120-541c-475a-9a6d-ac5a363d809f',
 '1cc5f016-9d45-4cf4-9afd-bee497d74e0d',
 'c3440dc1-4f35-4e8d-9e9e-4b96799930de',
 '682a1448-48af-4e9f-b49c-d3df85b21cac',
 '41cd7cf9-8faa-4084-b5df-136beb0cbfc6',
 '913a6778-81ca-4cd8-93fe-c61e32fbb9b7',
 '7695d7e7-e959-48aa-ae53-10df5455b986',
 'e3f126a3-46e1-4bb1-9cca-54f7a0933800',
 'fdda1065-4317-4e3a-896a-a576420a9a8e',
 '0a191bc2-d873-4144-bfc8-834f1bbe2b14',
 '5855dbbc-2d13-46dd-a8ba-f8f96462bc5f',
 '33734cf5-20b8-40c7-b604-7cf271ab978c',
 'c0855b95-8824-487e-98ea-ddd4d05756ac',
 '6ed8b968-0044-419b-b520-eae70e45ab4e',
 '5e73c56a-6438-4f8c-b374-847b72cddeb6',
 '8e08f1fc-91cb-4304-966a-2a373afd17e5',
 'bca053ac-3cfa-43bc-bc86-9115a39f6da7',
 '456449d5-b90b-48e5-9e83-a5766a244a3c',
 'e035ba92-58dd-47be-8bb6-64a4f7c310df',
 '2904c71d-c60a-4413-8a98-52810d04183c',
 'a3a96d76-de90-4298-bc0c-1e618543e18e',
 'deae2c3e-3741-4681-9d98-d9f3bf7838a0',
 '943b1825-cb7a-4dac-9146-0f12948450a9',
 '1283f44f-93bf-

#### Khởi tạo FAISS Vector Store

In [9]:
# Initialize the FAISS vector store
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

index = faiss.IndexFlatL2(len(hf_embeddings.embed_query("hello world")))
vector_db_path = 'vectorstore/db_faiss'

c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


#### Lưu các chunks vector vào Faiss (Chỉ chạy khi chưa lưu dữ liệu)

In [21]:
vector_store = FAISS(
    embedding_function=hf_embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [35]:
vector_store.add_documents(documents=list_of_chunks, ids=uuids)

['3032c2a5-668d-4f60-99c5-19e5af23b179',
 'd2833ebf-08f8-41cd-ad94-07eb5b0d9772',
 'e7e19c45-9735-4eea-a29e-3ccb0a3ba61e',
 'edccd22f-9218-42a0-a76b-ce1ea4b6a75e',
 '6eb70190-62f7-4b0a-97fb-4250aba30653',
 '84c556aa-8e5e-4917-a866-91c2344a60e6',
 'bb3e7505-f08b-4157-8193-5ad2a15b0e49',
 'bf98536b-0843-4ef3-a5b1-c08bcdf8d6e7',
 'b12e3c4b-3fa2-4772-8d1e-c3dd7885bec0',
 'b4cfe9fd-a5ad-486c-b485-f9b5b5a2a0bd',
 'a2329bcf-098d-4750-aae8-2d07a31910dc',
 'd8488273-cdd7-478f-b088-e08371b3587a',
 'f3e75721-2241-4e3a-a2e6-9af1b1a06c7d',
 '936c3001-6336-4d31-a96c-9099385a44b6',
 '20c75081-8d35-48f9-abc9-d805fc94f2dc',
 '20c5a819-a37d-4d6b-846f-719b3c56c85b',
 'b112866a-7d5b-4ff4-a5bd-4d38f2e43e62',
 '125236ee-b26f-4ffd-911d-f44f8d17f615',
 '84507f3d-2a3a-4a4a-a845-e61bc7155d03',
 '92a81126-fe04-43f0-b652-1d3f60858709',
 '8a8cae83-b755-4cd3-bbca-fbf762e385bc',
 'b64b4de7-cc61-4fe8-9bc8-41c4b0d0014a',
 '3a92666d-4b55-48f8-8cb0-36ea3f97f260',
 '6bee99ff-3cbe-43cb-8f6b-f56f0a425de5',
 '9cf27282-42f8-

In [48]:
# Lưu database vector
vector_store.save_local(vector_db_path)

#### Tải cơ sở vector Faiss (Chỉ chạy khi đã lưu dữ liệu)

In [10]:
# Tải lại FAISS index từ file
vector_store = FAISS.load_local(vector_db_path, hf_embeddings, allow_dangerous_deserialization = True)

In [11]:
print(vector_store.index_to_docstore_id)  # In ra mapping để kiểm tra

{0: 'a565a92f-12a3-4b70-9505-6540065cd7fe', 1: 'd902f774-78c4-41a2-b0c9-84441f42c6a7', 2: '4b82495c-c545-4709-90bf-6f8879a2d156', 3: '0df9ebd5-ee04-4612-972e-f9961433da9a', 4: 'd29c0f49-1d24-4c48-8c91-c3cac6c62c6c', 5: '01e09bad-b179-45ae-b88b-a6ac43ec0489', 6: '4a1d874a-889e-4710-8082-2e2c8c769eb8', 7: '292d423d-2972-45fb-83ec-37b88a846772', 8: 'bf0422f6-9124-46c0-b44b-619b6ba69265', 9: 'd8903da2-7219-42f1-a9c0-23fc59e45db0', 10: '47ae7d7e-e8f2-40de-bf7b-64cf75bc5c4a', 11: '225513c1-b82e-4834-9a43-78e5f17a652a', 12: '0fb7108d-0ea0-4aaa-86a1-32e29ba01a2b', 13: 'bf3a37be-a779-4950-837f-706d9d92d362', 14: 'e8dfbd6b-31fd-419e-855d-ffca54a725c9', 15: 'd552f7ca-4f93-4a4c-8270-7e1fe34616b5', 16: '802f1a16-ca41-4151-a738-a9053add861b', 17: 'ecfa80a4-ad37-4d83-a904-95bc8279077a', 18: 'f7a692df-1c73-4533-8753-67bad4f1f44d', 19: '4b5ee30a-aee6-43a1-9a5e-d44bda86dd4e', 20: 'f4a07fdf-4673-4c30-96d3-d2916facd137', 21: '770c5e4f-6cce-48d9-b6f9-cd5b586b7971', 22: '1a24ab95-e85e-4cf7-8d4a-ee50b4c4a9ab

### III. Các kỹ thuật retriever

#### Kỹ thuật vector search của các semantic chunks

In [12]:
# Similarity search example with the vector store
semantic_results = vector_store.similarity_search(
    "Nghiệm pháp Apley",
    k=10, 
)
for res in semantic_results:
    print(f"* {res.page_content} [{res.metadata}]")

* Ý NGHĨA Một vài nghiên cứu không đồng nhất đã được hoàn thành. Kết quả tổng hợp của hệ thống bảy nghiên cứu cho thấy độ nhạy của nghiệm pháp là 60.7% và độ đặc hiệu là 70,2% với một tỷ lệ không đồng đều HÌNH 1.2 Nghiệm pháp Apley 3.4, Làm nghiệm pháp xoay của Apley  [{'parent': 6}]
* Nghiệm pháp Tinel không đặc hiệu cho Ý NGHĨA tôn giáo thương thần kinh, thần kinh giữa Một một nghiên cưu đánh giá cho rằng nhưng cảm giác kim châm phát sinh từ tổn nghiệm pháp Tinel rất khó để phân biệt thương những người hội chứng ống cổ tay với 61 những người không bị Nghiêncưucho  [{'parent': 52}]
* Âm tính nếu nghiệm pháp tạo ra chỉ đau. Ý NGHĨA Một thử nghiệm hợp lý cho sự mất ổn định  [{'parent': 9}]
* MÔ TẢ thương SLAP bằng nghiệm pháp bàn tay Bệnh nhân ngồi hoặc đứng, khửu tay mở ngửa nghe có vẻ có hệ thống. Các bằng góc 60° và lòng bàn tay hướng lên. Bệnh chứng cho thấy rằng nghiệm pháp bàn tay nhân cố gắng nâng cánh tay chống lại lực ngửa tốt hơn một chút so với nghiệm pháp cản của người khám.

#### BM25 Retriever

In [13]:
# Sử dụng BM25 retriever từ LangChain
from langchain_community.retrievers import BM25Retriever

In [18]:
# bm25_params = {
#     "k1":1.25,
#     "b":0.5
# }

In [14]:
# Khởi tạo BM25 retriever với tham số tìm kiếm top 10 các kết quả liên quan nhất
bm25_retriever = BM25Retriever.from_documents(
  list_of_chunks, k = 10
)

In [15]:
bm25_results = bm25_retriever.invoke("Nghiệm pháp Apley")
bm25_results

[Document(metadata={'parent': 6}, page_content='Ý NGHĨA Một vài nghiên cứu không đồng nhất đã được hoàn thành. Kết quả tổng hợp của hệ thống bảy nghiên cứu cho thấy độ nhạy của nghiệm pháp là 60.7% và độ đặc hiệu là 70,2% với một tỷ lệ không đồng đều HÌNH 1.2 Nghiệm pháp Apley 3.4, Làm nghiệm pháp xoay của Apley '),
 Document(metadata={'parent': 7}, page_content='MÔ TẢ Thực hiện bằng cách yêu cầu bệnh nhân xác định và ‘cào’ vào xương bả vai đối diện, cả hai phía từ phía trên và phía dưới. Đau, hạn chế hoặc không đối xứng khithực hiện các động tác này có thể được coi là “dương tính”. NGUYÊN NHÂN Phổ biến Nhiều thể của sai khớp vai sẽ gây đau trong quá trình làm nghiệm pháp cào lưng này \nCƠ CHẾ Nghiệm pháp được thực hiện để đánh giá phạm vi hoạt động của khớp vai, cụ thể, phạm vi di chuyển của cánh tay, tầm hoạt động vào trong hay ra ngoài của khớp vai. Mặc dù nó gợi ý đến bệnh lý về dây chằng khớp vai, tổn thương cơ trên vai, nhưng hầu hết tổn thương bao khớp, dây chằng, cơ hay xương t

#### BGE-m3 Reranker

In [16]:
# Lấy kết quả chunks từ BM25 Retriever và FAISS vector search
content = set()
retrieval_docs = []

for result in semantic_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

for result in bm25_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

len(retrieval_docs)

16

In [17]:
# Định nghĩa max_length cho padding
max_length = 512  # Hoặc bạn có thể thay đổi giá trị này tùy vào yêu cầu

# Tokenize the text with padding and truncation (Mã hóa dữ liệu)
encoded_inputs = tokenizer(
    "Nghiệm pháp Apley là một bài kiểm tra trong y học.",  # văn bản muốn token hóa
    padding='max_length',    # Đảm bảo tất cả các văn bản đều có cùng độ dài
    truncation=False,         # Cắt văn bản nếu quá dài
    max_length=max_length,   # Chiều dài tối đa
    return_tensors="pt"      # Trả về kết quả dưới dạng tensors (nếu cần)
)

In [18]:
# Sử dụng mô hình BAAI/bge-reranker-v2-m3 để xếp hạng các kết quả chunks dựa trên số điểm tính toán
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Thiết lập use_fp16 = True để tăng tốc tính toán với một sự giảm hiệu suất nhẹ.
pairs = [["Nghiệm pháp Apley",doc.page_content] for doc in retrieval_docs]
score = reranker.compute_score(pairs,normalize = True)
score

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 133.26it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


[0.9276259821794456,
 0.0019494021275301956,
 0.00455611371445455,
 0.01005236865796813,
 0.12721826766848623,
 0.0773781063026379,
 0.04256272864849986,
 0.004382336640730745,
 0.02479816019558566,
 0.0012255674483830521,
 0.9628132248397597,
 0.006641112642077409,
 0.5740366277485736,
 0.016029147365901313,
 0.01536534144130222,
 0.04577727258077672]

#### Smart Retriever

In [ ]:
# Xây dựng, tổng hợp các kỹ thuật truy vấn nâng cao thành lớp 'Retriever'
class Retriever:
  def __init__(self, semantic_retriever, bm25_retriever, reranker):
    self.semantic_retriever = semantic_retriever
    self.bm25_retriever = bm25_retriever
    self.reranker = reranker

  def __call__(self,query):
    semantic_results = self.semantic_retriever.similarity_search(
      query,
      k=10,
    )
    bm25_results = self.bm25_retriever.invoke(query)

    content = set()
    retrieval_docs = []

    for result in semantic_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    for result in bm25_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    pairs = [[query,doc.page_content] for doc in retrieval_docs]

    scores = self.reranker.compute_score(pairs,normalize = True)

    # Lấy tài liệu nguồn từ phần tử con dựa trên điểm số ngưỡng
    context_1 = []
    context_2 = []
    context = []
    parent_ids = set()
    for i in range(len(retrieval_docs)):
      # Điểm liên quan >= 0.6 sẽ được sử dụng làm kiểu ngữ cảnh 1 (chỉ ra sự liên quan cao hơn đối với truy vấn).
      if scores[i] >= 0.6:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_1.append(documents[parent_idx])
      # Điểm liên quan >= 0.1 sẽ được sử dụng làm kiểu ngữ cảnh 2 (chỉ ra sự liên quan trung bình đến thấp đối với truy vấn).
      elif scores[i] >= 0.1:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_2.append(documents[parent_idx])
      
    if len(context_1) > 0:
      print('Context 1')
      context=context_1
    elif len(context_2) > 0:
      print('Context 2')
      context=context_2
    else:
      # Nếu điểm liên quan < 0.1, điều này chỉ ra rằng không có tài liệu liên quan.
      print('No relevant context')
    return context

In [20]:
# Kiểm tra bộ truy xuất Smart Retriever
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker)
context = retriever("Nghiệm pháp Apley")
context

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 334.45it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Context 1


[Document(metadata={'title': 'Nghiệm pháp Apley', 'table': []}, page_content='MÔ TẢ không phải là một nghiệm pháp chẩn đoán Bệnh nhân nằm sấp, đầu gối gấp 90°, tiến đặc biệt hữu ích của chấn thương sụn hành ép mạnh vào gót chân từ trên xuống chêm. Nhưng những phát hiện đã được dưới, ép xương chày xuống xương đùi. Sau nảy sinh trong kết quả của một nghiên cứu gộp. Ngoài ra, nhiều người khám cũng đó người thực hiện tiến hành xoay xương không thực hiện được nghiệm pháp xoay chày vào trong hoặc ra ngoài. của Apley do có thể gây đau dữ dội nếu xuất hiện kèm theo một chấn thương. \nNGUYÊN NHÂN • Chấn thương sụn chêm \nCƠ CHẾ Áp lực lớn từ xương chày lên xương đùi sẽ được nhằm vào khe khớp tại vị trí gây tổn thương sụn chêm. Nếu có tổn thương trực tiếp có thể gây nên đau. \nÝ NGHĨA Một vài nghiên cứu không đồng nhất đã được hoàn thành. Kết quả tổng hợp của hệ thống bảy nghiên cứu cho thấy độ nhạy của nghiệm pháp là 60.7% và độ đặc hiệu là 70,2% với một tỷ lệ không đồng đều HÌNH 1.2 Nghiệm phá

### IV. Tải mô hình và xây dựng câu trả lời

#### Tải thư viện

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.ctransformers import CTransformers
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import BitsAndBytesConfig

#### Tải mô hình đã fine-tuned 

In [38]:
# Đường dẫn tới mô hình và tokenizer
model_path = "../mlperf_env/mlperf_env/model/4BIT00006"

# Tham số lượng tử hóa mô hình
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", False 
device_map = {"": 0}
# Tải tokenizer và mô hình với tham số QLoRA 
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
# Thiết lập pad token thành eos_token (có 2 trường hợp)
    ## Lựa chọn 1: Thiết lập eos_token thành pad_token
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token 

    # Hoặc, lựa chọn 2: Thêm một padding token mới
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map=device_map
    )

# model.config.use_cache = False
# model.config.pretraining_tp = 1

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.65s/it]


#### Tạo Prompt Template và QA Question

In [39]:
system_prompt = f"""
Bạn là một trợ lí ảo MediChat ViVi nhiệt tình và trung thực. 
Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn. 
Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, 
hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác, vui lòng không chia sẻ thông tin sai lệch."""

In [40]:
# Hàm tạo câu trả lời từ mô hình Llama
def generate_answer_from_documents(query, context, model, tokenizer):
    formatted_prompt = f"""
        Câu hỏi của người dùng: {query}
        Trả lời câu hỏi dựa vào các thông tin sau: {context}"""
    
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": formatted_prompt}]
    # Mã hóa prompt 
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        # padding='max_length',  # Padding đến độ dài tối đa
        truncation=True,       # Cắt ngắn nếu cần thiết
        return_tensors="pt"
    ).to(model.device)
    
    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    # Tạo câu trả lời từ mô hình
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Sử dụng attention mask
        max_new_tokens=512,
        min_length=30, 
        do_sample=True, # Quyết định có sử dụng kỹ thuật sampling ngẫu nhiên hay không (True là có, False là không).
        temperature=0.7,    # Điều chỉnh mức độ ngẫu nhiên trong quá trình sinh câu trả lời (giá trị thấp hơn -> ít ngẫu nhiên).
        top_p=0.9,  # Giới hạn tỷ lệ xác suất của các token sẽ được chọn
        min_p=0.1,
        eos_token_id=terminators
    )

    # Giải mã và trả về câu trả lời
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

# Kiểm tra với Retriever 
def get_answer_with_retriever(query, retriever, model, tokenizer):
    # Sử dụng retriever để lấy context (dữ liệu từ các tài liệu liên quan)
    context = retriever(query)

    # Nếu context có dữ liệu, sử dụng mô hình để tạo câu trả lời
    if context:
        context_str = "\n".join([doc.page_content for doc in context])  # Chuyển tất cả các tài liệu thành chuỗi
        return generate_answer_from_documents(query, context_str, model, tokenizer)
    else:
        return "Không tìm thấy thông tin liên quan để trả lời câu hỏi."

#### Thử nghiệm thực tế câu hỏi

In [41]:
query = "Nghiệm pháp Apley là gì?"
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker)

# Gọi hàm để lấy câu trả lời từ retriever và mô hình Llama
answer = get_answer_with_retriever(query, retriever, model, tokenizer)
print("Answer:", answer)

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 97.80it/s]
c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\TLCN_HealthChatbot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Context 1
Answer: Nghiệm pháp cào lưng của Apley là một phương pháp đánh giá chức năng của khớp vai, cụ thể là phạm vi di chuyển của cánh tay, tầm hoạt động vào trong hay ra ngoài của khớp vai. Nó được thực hiện bằng cách yêu cầu bệnh nhân xác định và 'cào' vào xương bả vai đối diện, cả hai phía từ phía trên và phía dưới. Nếu đau, hạn chế hoặc không đối xứng khi thực hiện các động tác này có thể được coi là "dương tính". Phương pháp này thường được sử dụng để đánh giá nhiều thể sai khớp vai và gợi ý đến bệnh lý về dây chằng khớp vai, tổn thương cơ trên vai, nhưng hầu hết tổn thương bao khớp, dây chằng, cơ hay xương tại khớp vai đều có kết quả dương tính.
